# 基本的な使い方

## 使用するデータの準備
[make_dataset.ipynb](./make_dataset.ipynb)で作ったlivedoorニュースのデータセットをロードしタイトルを抽出します。

In [2]:
import pandas as pd

df = pd.read_csv("./livedoor-news.csv")

## タイトル文章のベクトル化
文章をベクトル化する方法はいくつかありますが、有料のOpenAI Embeddings APIを使う方法と、無料のsentence-transformersを使う方法を紹介します。

### OpenAI Embeddings APIを使う場合
OpenAIのアカウントを作成して[こちら](https://platform.openai.com/api-keys)でapi-keyを発行し以下の行の右辺のsk-XXXを書き換えてください。前後にダブルクオートは入れないでください。
今回の8000弱の短い文章のリクエストだとトークン数は30万弱で費用は$0.03程度でした。時間は20分以上かかりました。

In [ ]:
%env OPENAI_API_KEY=sk-XXX

In [5]:
import os

import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
client = openai.OpenAI()


def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [6]:
# 次元数
model = "text-embedding-ada-002"
len(get_embedding("携帯の使い方が難しい", model))

1536

In [11]:
from tqdm.notebook import tqdm

tqdm.pandas()

df["title_embedding"] = df["title"].progress_apply(lambda x: get_embedding(x, model))

In [ ]:
# fileから読み込んだあともembeddingが復元できるよう加工して保存しています
w_df = df.copy()
w_df["title_embedding"] = w_df["title_embedding"].apply(list)
w_df.to_csv("./livedoor-news-with-title-embedding-openai.csv", index=False)

### sentence-transformersを使う場合
この例では多言語対応モデルを使います。

In [8]:
from sentence_transformers import SentenceTransformer

# CPUを使用する場合
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# GPUを使用する場合
# model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda')

In [9]:
def get_embedding(text, model):
    return model.encode(text)

In [10]:
# 次元数
len(get_embedding("携帯の使い方が難しい", model))

768

In [11]:
df["title_embedding"] = df["title"].apply(lambda x: get_embedding(x, model))

In [12]:
# fileから読み込んだあともembeddingが復元できるよう加工して保存しています
w_df = df.copy()
w_df["title_embedding"] = w_df["title_embedding"].apply(list)
w_df.to_csv("./livedoor-news-with-title-embedding-st.csv", index=False)

## Valdクラスタの準備
こちらの[記事](https://zenn.dev/vald/articles/01-get-started-with-chive)を参考にValdクラスタを構築してください。

values.yaml の agent.ngt.dimension には、実際に挿入するベクトルの次元数(OpenAI Embeddings APIを使う場合は1536、
sentence-transformersを使う場合は768)を設定してください。agent.ngt.distance_type には l2 を設定してください。

In [31]:
import numpy as np
import pandas as pd

df = pd.read_csv("./livedoor-news-with-title-embedding-openai.csv")
# df = pd.read_csv('./livedoor-news-with-title-embedding-st.csv')

In [32]:
df["title_embedding"] = df["title_embedding"].apply(eval).apply(np.array)

In [78]:
import grpc
import numpy as np
from vald.v1.payload import payload_pb2
from vald.v1.vald import search_pb2_grpc, upsert_pb2_grpc

In [79]:
## 接続先Host名(Host:Port)
host = "localhost:80"

## 次元数
dimension = 1536  # OpenAI Embeddings APIを使う場合
# dimension = 768  # sentence-transformersを使う場合

In [80]:
channel = grpc.insecure_channel(host)

ベクトルの挿入ができるか試してみます。

In [81]:
usstub = upsert_pb2_grpc.UpsertStub(channel)

In [82]:
vec = payload_pb2.Object.Vector(id="0", vector=df["title_embedding"][0])
uscfg = payload_pb2.Upsert.Config(skip_strict_exist_check=True)
usstub.Upsert(payload_pb2.Upsert.Request(vector=vec, config=uscfg))

name: "vald-agent-ngt-1"
uuid: "0"
ips: "127.0.0.1"

挿入したベクトルが検索できるか試してみます。データ挿入後、index作成が終わらないと検索結果に反映されないので数分待って実行してください。

In [88]:
sstub = search_pb2_grpc.SearchStub(channel)

In [91]:
svec = np.array([0.01] * dimension, dtype="float32")  # クエリ用のテストベクトル
scfg = payload_pb2.Search.Config(num=10, radius=-1.0, epsilon=0.01, timeout=3000000000)
sstub.Search(payload_pb2.Search.Request(vector=svec, config=scfg))

results {
  id: "0"
  distance: 1.08456945
}

## ニュースの全タイトルをValdに挿入
挿入完了後もindex作成が終わるまでは検索結果に反映されないのでさらに数分待ってから検索してください。

In [ ]:
from tqdm.notebook import tqdm

for row in tqdm(df.itertuples(), total=len(df)):
    vec = payload_pb2.Object.Vector(id=str(row.Index), vector=row.title_embedding)
    uscfg = payload_pb2.Upsert.Config(skip_strict_exist_check=True)
    usstub.Upsert(payload_pb2.Upsert.Request(vector=vec, config=uscfg))

## 任意のクエリに類似したニュースタイトルを検索

In [93]:
def get_search_response(text, model, k):
    qvec = get_embedding(text, model)
    scfg = payload_pb2.Search.Config(
        num=k, radius=-1.0, epsilon=0.01, timeout=3000000000
    )
    return sstub.Search(payload_pb2.Search.Request(vector=qvec, config=scfg))

In [94]:
def display_results_top_k(text, model, k):
    response = get_search_response(text, model, k=k)
    for result in response.results:
        rtitle = df["title"][int(result.id)]
        rdistance = result.distance
        print(f"title: {rtitle}, distance: {rdistance}")

In [100]:
text = "携帯の使い方が難しい"
display_results_top_k(text, model, k=3)

title: 携帯の英字入力で「たかはし」と打つと「GAME」になる, distance: 0.5004885196685791
title: やっぱり使いにくい！スマホからガラケーに戻す人が多いらしい!?【話題】, distance: 0.5063361525535583
title: スマートフォンを使う上での最低限のマナーとは, distance: 0.5123698711395264


In [101]:
# 英語でのクエリも行えます
text = "It's difficult to use cell phone."
display_results_top_k(text, model, k=3)

title: やっぱり使いにくい！スマホからガラケーに戻す人が多いらしい!?【話題】, distance: 0.569595992565155
title: ドコモ障害の原因はスマホだった？普及が進むスマホに課題か【話題】, distance: 0.6260550618171692
title: 本当に電話もメールもらくらく操作なの？「らくらくスマートフォンF-12D」の電話機能をチェック【レビュー】, distance: 0.6301419138908386


In [102]:
text = "今年のホームラン王は誰か"
display_results_top_k(text, model, k=3)

title: 【SportsWatch】2010年プロ野球界のキーマンは？, distance: 0.5248281359672546
title: 【SportsWatch】日本代表再建のキーマンは？, distance: 0.5396592020988464
title: 五輪サッカー日韓戦において、韓国が注目する“ある選手”とは？, distance: 0.5547349452972412


## ベクトル検索エンジンを使わない場合
アプリケーション側で計算することでもベクトルの距離計算を行うことができます。ですがValdを使えばデータ量が増えても高速な検索を行うことができます。

Valdは近似近傍検索なので精度面が心配になるかもしれません。そこで正確な計算結果とどれくらい差があるのかnumpyを使った例を使って比較してみます。

Valdのパラメータで精度と速度のトレードオフを調整でき、今回はagent.ngt.creation_edge_size=20、agent.ngt.search_edge_size=40を設定しています。

In [103]:
def display_top_k_with_numpy(text, df, k):
    insert_features = np.array([x for x in df["title_embedding"].values])
    query_feature = get_embedding(text=text, model=model)
    distances = np.linalg.norm(
        query_feature - insert_features, axis=1
    )  # distance_type=L2と等価
    distance_indexes = np.argsort(distances)[:k]

    for idx in distance_indexes:
        print(f"title: {df['title'][int(idx)]}, distance: {distances[int(idx)]}")

In [104]:
text = "携帯の使い方が難しい"
display_top_k_with_numpy(text, df, k=3)

title: 携帯の英字入力で「たかはし」と打つと「GAME」になる, distance: 0.5004885569201721
title: やっぱり使いにくい！スマホからガラケーに戻す人が多いらしい!?【話題】, distance: 0.5063361694492042
title: スマートフォンを使う上での最低限のマナーとは, distance: 0.5123698822775186


distanceを見るとValdを使った時の結果とほとんど同じになってることがわかるかと思います。

## 謝辞
livedoorニュースの記事は以下ライセンスに基づき改変せず使用させていただきました。

https://creativecommons.org/licenses/by-nd/2.1/jp/

ニュース記事を公開いただきました、独女通信、家電チャンネル、MOVIE ENTER、エスマックス、トピックニュース、ITライフハック、livedoor HOMME、Peachy、Sports Watch様に感謝を申し上げます。